In [7]:
# ┌──── Python: 데이터 처리 & 시각화 ────┐
import glob, os
import pandas as pd
import numpy as np
import holidays
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from pathlib import Path
font_path = "/System/Library/Fonts/AppleSDGothicNeo.ttc"
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

# Get Data

In [8]:
DATA_DIR = 'data'

train_raw = pd.read_csv(f'{DATA_DIR}/train/train.csv')

# 모든 test_*.csv 순회
test_raw_list = []
test_files = sorted(glob.glob(os.path.join(DATA_DIR, 'test/TEST_*.csv')))
for path in test_files:
    test_df = pd.read_csv(path)
    test_raw_list.append(test_df)

submission = pd.read_csv(f"{DATA_DIR}/submission/sample_submission.csv")

# EDA

### 영업일자
- train: 2023-01-01 ~ 2024-06-15
- test: 2024-06-16 ~ 2024-05-24 (28일씩 10개의 파일)

In [9]:
def raw_to_dataframe(df):
    df = df.pivot(
        index='영업일자',
        columns='영업장명_메뉴명',
        values='매출수량'
    )
    df = df.sort_index()
    # 컬럼 축 이름(영업장명_메뉴명) 제거
    df = df.rename_axis(columns=None)

    # 인덱스를 컬럼으로 내리고
    df = df.reset_index()
    return df

train = raw_to_dataframe(train_raw)
test_list = [raw_to_dataframe(test_raw) for test_raw in test_raw_list]

In [10]:
def add_date_features(df, date_col: str = '영업일자') -> pd.DataFrame:
    """
    df 에 아래 3개 컬럼을 추가해서 리턴합니다.
      - weekday: 한국어 요일명
      - is_holiday: 주말+한국 공휴일 여부 (bool)
      - holiday_streak: 연속 공휴일 개수 (연속 블록 내 모든 날에 블록 길이)
    
    df는 date_col이 datetime 형식이거나, parseable한 문자열이어야 합니다.
    """
    # 1) 날짜 컬럼을 datetime 으로 변환
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    
    # 2) 한국어 요일명 매핑
    kor_weekdays = {
        0: '월요일',
        1: '화요일',
        2: '수요일',
        3: '목요일',
        4: '금요일',
        5: '토요일',
        6: '일요일',
    }
    df['weekday'] = df[date_col].dt.weekday.map(kor_weekdays)
    
    # 3) 공휴일 여부
    kr_holidays = holidays.KR()
    # is_holiday: 주말 또는 kr_holidays 에 포함된 날
    df['is_holiday'] = (
        df[date_col].dt.weekday.isin([5, 6])  # 토·일
        | df[date_col].isin(kr_holidays)      # 한국 공휴일
    )
    
    # 4) 연속 공휴일 수 계산
    #    공휴일 시퀀스가 끊길 때마다 그룹을 나누고, 각 그룹의 크기를 구함
    mask = df['is_holiday']
    # 그룹 번호: 연속 블록마다 다른 번호
    grp = (mask != mask.shift(1)).cumsum()
    # 블록 크기 계산
    block_sizes = mask.groupby(grp).transform('sum')
    # 연속 공휴일 수: 공휴일인 날에만 block_sizes, else 0
    df['holiday_streak'] = block_sizes.where(mask, other=0).astype(int)
    
    return df

In [11]:
add_date_features(train)

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림,weekday,is_holiday,holiday_streak
0,2023-01-01,0,0,9,2,0,0,0,0,0,...,0,0,0,0,0,0,0,일요일,True,1
1,2023-01-02,0,0,2,4,1,0,0,0,0,...,0,0,0,0,0,0,0,월요일,False,0
2,2023-01-03,0,0,2,3,0,2,10,0,0,...,0,0,0,0,0,0,0,화요일,False,0
3,2023-01-04,0,0,3,6,0,0,0,0,0,...,0,0,0,0,0,0,0,수요일,False,0
4,2023-01-05,0,19,6,2,0,0,8,0,0,...,0,0,0,0,0,0,0,목요일,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,2024-06-11,0,67,1,0,0,0,14,0,0,...,2,55,20,10,72,20,12,화요일,False,0
528,2024-06-12,0,94,0,0,0,0,19,0,1,...,13,62,26,12,65,7,10,수요일,False,0
529,2024-06-13,0,111,2,0,0,0,7,8,2,...,2,75,30,1,95,17,14,목요일,False,0
530,2024-06-14,16,122,2,0,0,0,45,4,9,...,6,69,59,6,69,13,12,금요일,False,0


In [16]:
def create_sliding_window(df: pd.DataFrame, input_window: int = 28, output_window: int = 7):
    """
    다변량 시계열 데이터에서 TabPFN 학습을 위한 (X, y) 쌍을 만드는 함수.
    X: input_window일치 시계열(flatten됨)
    y: output_window일치 이후 매출(flatten됨)

    Args:
        df (pd.DataFrame): '날짜가 인덱스인 다변량 시계열' DataFrame
        input_window (int): 입력 기간 (기본 28일)
        output_window (int): 예측 기간 (기본 7일)

    Returns:
        X (np.ndarray): (샘플 수, input_window * 변수 수)
        y (np.ndarray): (샘플 수, output_window * 변수 수)
    """
    df = df.copy()
    df = df.reset_index(drop=True)

    n_rows = df.shape[0]
    n_features = df.shape[1]

    X, y = [], []
    for start in range(n_rows - input_window - output_window + 1):
        end = start + input_window
        out_end = end + output_window

        x_window = df.iloc[start:end].values.flatten()
        y_window = df.iloc[end:out_end].values.flatten()

        X.append(x_window)
        y.append(y_window)

    return np.array(X), np.array(y)

# 예제용 더미 데이터프레임 생성
dates = pd.date_range("2022-01-01", periods=60)
data = pd.DataFrame({
    'item1': np.random.poisson(10, size=60),
    'item2': np.random.poisson(20, size=60),
    'item3': np.random.poisson(5, size=60),
}, index=dates)

X, y = create_sliding_window(train)

In [20]:
y.shape

(498, 1358)